In [1]:
import pandas as pd

In [39]:
cols_names = ['user_id', 'item_id', 'rating', 'timestamp']
main_df = pd.read_csv("data/u.data", sep = '\t', names = cols_names)

In [40]:
main_df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [41]:
movies = pd.read_csv("/data/Movie_Id_Titles.txt")
movies.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [42]:
final_df = pd.merge(main_df, movies, on='item_id')
final_df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [43]:
ratings = pd.DataFrame(final_df.groupby('title')['rating'].mean())
ratings['counts'] = final_df.groupby('title')['rating'].count()
ratings.head()

,rating,counts
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


#########################################

In [50]:
matrix = final_df.pivot_table(index='user_id',columns='title',values='rating')
matrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",8 1/2 (1963),8 Heads in a Duffel Bag (1997),8 Seconds (1994),A Chef in Love (1996),Above the Rim (1994),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Sea of Time (1995),Addams Family Values (1993),Addicted to Love (1997),"Addiction, The (1995)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Affair to Remember, An (1957)","African Queen, The (1951)",Afterglow (1997),"Age of Innocence, The (1993)",Aiqing wansui (1994),Air Bud (1997),Air Force One (1997),"Air Up There, The (1994)",Airheads (1994),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alaska (1996),Albino Alligator (1996),...,"Whole Wide World, The (1996)",Widows' Peak (1994),"Wife, The (1995)",Wild America (1997),Wild Bill (1995),"Wild Bunch, The (1969)",Wild Reeds (1994),Wild Things (1998),William Shakespeare's Romeo and Juliet (1996),Willy Wonka and the Chocolate Factory (1971),Window to Paris (1994),Wings of Courage (1995),Wings of Desire (1987),"Wings of the Dove, The (1997)",Winnie the Pooh and the Blustery Day (1968),"Winter Guest, The (1997)",Wishmaster (1997),With Honors (1994),Withnail and I (1987),Witness (1985),"Wizard of Oz, The (1939)",Wolf (1994),"Woman in Question, The (1950)","Women, The (1939)","Wonderful, Horrible Life of Leni Riefenstahl, The (1993)",Wonderland (1997),"Wooden Man's Bride, The (Wu Kui) (1994)","World of Apu, The (Apur Sansar) (1959)","Wrong Trousers, The (1993)",Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [376]:
import warnings
warnings.filterwarnings("ignore")

def find_similar_movies(name=matrix.columns[0]):
  movie_user_ratings = matrix[name]
  similar_to_movie = matrix.corrwith(movie_user_ratings)
  corr_movie = pd.DataFrame(similar_to_movie,columns=['Correlation'])
  corr_movie.fillna(0, inplace=True)
  corr_movie['movie_name'] = corr_movie.index
  corr_movie = corr_movie.reset_index(drop=True) 
  return corr_movie


def find_similar_users(user_id, k = 10):
  thresh = int(len(matrix.columns) * 0.2)
  test = matrix.dropna(thresh=thresh)
  user_movie_data = matrix.iloc[user_id]
  similar_to_user = test.T.corrwith(user_movie_data)
  corr_user = pd.DataFrame(similar_to_user,columns=['Correlation'])
  corr_user.fillna(0, inplace=True)
  corr_user = corr_user.sort_values('Correlation',ascending=False).head(k)
  corr_user['user_id'] = corr_user.index
  corr_user = corr_user.reset_index(drop=True) 
  return corr_user

In [386]:
def sparse(data, sparsity_thresh = 1):
  data = data.fillna(0)
  val = data.value_counts()
  if val[0] > len(data)*sparsity_thresh:
    return True
  return False

In [400]:
def cumulate_values(movies_data, user_data):
  movie_dict = {}

  i = 0
  for m, u in zip(movies_data, user_data):
    movie_dict[matrix.columns[i]] = (m + u)
    i += 1
  
  suggestion_df = pd.DataFrame(movie_dict.items(), columns = ['names', 'score']).sort_values('score',ascending=False).head(10)

  return suggestion_df

In [401]:
def best_general_recommendation():
  new_mat = ratings.sort_values('rating', ascending=False).sort_values('counts', ascending=False).head(10)
  return new_mat

In [409]:
# real working starts here

import numpy as np

def predictor(user_id = 2):
  global matirx

  if not sparse(matrix.iloc[user_id]):

    # -------------------------------------------------------------------------------
    k_user_data = find_similar_users(user_id, k = 3)
  
    combined_user_data = []
    for i in range(len(k_user_data)):
      each_user_id = k_user_data['user_id'][i]
      corr_value = k_user_data['Correlation'][i]
      watch_data_for_user = matrix.iloc[each_user_id] * corr_value    # dampening effect
      combined_user_data.append(watch_data_for_user)

    combined_user_data = np.array(combined_user_data)
    combined_user_data = np.nan_to_num(combined_user_data, copy=True, nan=0, posinf=None, neginf=None)
    user_data = np.sum(combined_user_data, axis = 0) / combined_user_data.shape[0]

    # ----------------------------------------------------------------------------------

    k_movies_data = []
    for i, each_movie in enumerate(matrix.iloc[user_id]):
      if not np.isnan(each_movie) and each_movie >= 3.0:
        corr_movies = find_similar_movies(matrix.columns[i])['Correlation']
        k_movies_data.append(corr_movies)

    k_movies_data = np.array(k_movies_data)
    k_movies_data = np.nan_to_num(k_movies_data, copy=True, nan=0, posinf=None, neginf=None)
    movies_data = np.sum(k_movies_data, axis = 0) /k_movies_data.shape[0]

    # ------------------------------------------------------------------------------------
    
    suggestion = cumulate_values(movies_data, user_data)
    suggestion['norm_score'] = suggestion['score']/np.max(suggestion['score'])
    return suggestion
  else:
    print("you havent watched much movies right now, thus can't figure out your taste\nHere are few suggestions for you!")
    return best_general_recommendation()

In [ ]:
suggestions = predictor(2)
suggestions

In [404]:
suggestions = predictor(32)
suggestions

,names,score,norm_score
1190,Pulp Fiction (1994),2.775802,1.000000
1500,Titanic (1997),2.657907,0.957528
1398,Star Wars (1977),2.629341,0.947236
667,Heathers (1989),2.625342,0.945796
996,Monty Python and the Holy Grail (1974),2.624103,0.945350
612,"Godfather, The (1972)",2.610971,0.940619
1281,Schindler's List (1993),2.603739,0.938013
1178,"Princess Bride, The (1987)",2.596178,0.935289
317,"Clockwork Orange, A (1971)",2.574585,0.927510
1657,Young Frankenstein (1974),2.500693,0.900890


In [405]:
suggestions = predictor(87)
suggestions

,names,score,norm_score
780,Jerry Maguire (1996),2.528881,1.000000
1394,Star Trek: First Contact (1996),2.210323,0.874032
744,Independence Day (ID4) (1996),2.117946,0.837504
463,Eraser (1996),2.090443,0.826628
333,Contact (1997),2.041539,0.807289
987,Mission: Impossible (1996),1.991284,0.787417
1270,Sabrina (1995),1.905086,0.753332
1297,Sense and Sensibility (1995),1.825369,0.721809
389,Dead Man Walking (1995),1.801226,0.712262
1495,"Time to Kill, A (1996)",1.777146,0.702740


In [406]:
suggestions = predictor(341)
suggestions

,names,score,norm_score
1398,Star Wars (1977),5.061844,1.000000
1204,Raging Bull (1980),5.028805,0.993473
7,2001: A Space Odyssey (1968),4.921619,0.972298
1572,"Usual Suspects, The (1995)",4.910186,0.970039
1317,"Shawshank Redemption, The (1994)",4.886926,0.965444
612,"Godfather, The (1972)",4.861222,0.960366
1205,Raiders of the Lost Ark (1981),4.838801,0.955936
1208,Raising Arizona (1987),4.726977,0.933845
456,"Empire Strikes Back, The (1980)",4.664679,0.921537
1456,Taxi Driver (1976),4.652356,0.919103


In [407]:
suggestions = predictor(69)
suggestions

,names,score,norm_score
1317,"Shawshank Redemption, The (1994)",3.240424,1.000000
448,Eat Drink Man Woman (1994),3.209164,0.990353
1217,Rebel Without a Cause (1955),3.204221,0.988828
1398,Star Wars (1977),3.196255,0.986369
773,Jaws (1975),3.192866,0.985323
623,GoodFellas (1990),3.169505,0.978114
612,"Godfather, The (1972)",3.162133,0.975839
1189,Psycho (1960),3.154927,0.973616
273,Casablanca (1942),3.140922,0.969293
303,Citizen Kane (1941),3.089291,0.953360
